In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install -U tensorflow==2.0.0
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import tensorflow
import os

        
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image  import ImageDataGenerator
from tensorflow.keras.utils  import  img_to_array,load_img
import matplotlib.pyplot as plt
from glob import glob

In [6]:
train_path = "/kaggle/input/fruits/fruits-360/Training/"
test_path = "/kaggle/input/fruits/fruits-360/Test/"

In [ ]:
img = load_img("/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Training/apple_golden_1/r0_100.jpg")
plt.imshow(img)
plt.title("Apple Golden")
plt.axis("off")
plt.show()

In [ ]:
shape_of_image = img_to_array(img)
print(shape_of_image.shape)

In [ ]:
classes = glob("/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Training"+ "/*")
number_of_class = len(classes)
print("Number of class : " , number_of_class)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                   shear_range = 0.3,
                   horizontal_flip = True,
                   zoom_range = 0.3)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory("/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Training",
                                                   target_size = shape_of_image.shape[:2],
                                                   batch_size = 32,
                                                   color_mode = 'rgb',
                                                   class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory("/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Test",
                                                   target_size = shape_of_image.shape[:2],
                                                   batch_size = 32,
                                                   color_mode = 'rgb',
                                                   class_mode = 'categorical')

In [11]:
model = Sequential()

In [12]:
model.add(Conv2D(32,(3,3),activation = 'relu', input_shape = shape_of_image.shape))
model.add(MaxPooling2D())

model.add(Conv2D(32,(3,3),activation = 'relu', input_shape = shape_of_image.shape))
model.add(MaxPooling2D())

model.add(Conv2D(64,(3,3),activation = 'relu', input_shape = shape_of_image.shape))
model.add(MaxPooling2D())

In [13]:
model.add(Flatten())
model.add(Dense(1024,activation='relu'))

In [14]:
model.add(Dropout(0.5))
model.add(Dense(number_of_class,activation = 'softmax'))

In [15]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

In [16]:
batch_size = 32
number_of_batch = 1600 // batch_size

In [ ]:
hist = model.fit_generator(
    generator = train_generator,
    steps_per_epoch = number_of_batch,
    epochs = 100,
    validation_data = test_generator,
    validation_steps = 800 // batch_size
                   )

In [ ]:
model.save_weights("trial.h5")

In [ ]:
print(hist.history.keys())
plt.plot(hist.history["loss"],label = "Train Loss")
plt.plot(hist.history["val_loss"],label = "Validaton Loss")
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(hist.history["accuracy"],label = "Train Accuracy")
plt.plot(hist.history["val_accuracy"],label = "Validaton Accuracy")
plt.legend()
plt.show()

In [ ]:
import json
with open("traial.json","w") as f:
    json.dump(hist.history,f)

In [ ]:
import codecs 
with codecs.open("traial.json","r",encoding = "utf-8") as f:
    h = json.loads(f.read())